In [ ]:
import requests
import json
from shapely.geometry import shape, mapping
from shapely.ops import unary_union

# 1. Setup
REPO_API = "https://api.github.com/repos/idoivri/israel-municipalities-polygons/git/trees/master?recursive=1"
RAW_BASE = "https://raw.githubusercontent.com/idoivri/israel-municipalities-polygons/master"

# Cities that MUST remain High Quality (No simplification)
HD_CITIES = [
    "jerusalem_yerushalayim", "tel_aviv_yafo", "haifa", "rishon_letsiyon",
    "petah_tikva", "ashdod", "netanya", "beer_sheva", "bnei_brak", "bat_yam", "holon"
]

print("1. Scanning repository...")
r = requests.get(REPO_API)
files = [x["path"] for x in r.json().get("tree", []) if x["path"].endswith(".geojson")]

features = []
border_geoms = []

print(f"2. Processing {len(files)} municipalities...")

for i, path in enumerate(files):
    raw_id = path.split("/")[-1].replace(".geojson", "").lower()

    try:
        resp = requests.get(f"{RAW_BASE}/{path}")
        if resp.status_code != 200: continue

        data = resp.json()
        all_geoms = [shape(feat["geometry"]) for feat in data["features"]]
        combined_geom = unary_union(all_geoms)
        # LOGIC:
        # If it's a major city -> Keep it sharp (0.00001 tolerance)
        # If it's a small town -> Simplify it (0.0005 tolerance) to save space
        if raw_id in HD_CITIES:
            simplified = combined_geom.simplify(0.00001, preserve_topology=True)
        else:
            simplified = combined_geom.simplify(0.0005, preserve_topology=True)

        if not simplified.is_empty:
            border_geoms.append(simplified)

            new_feat = {
                "type": "Feature",
                "id": raw_id,
                "properties": {"name": raw_id},
                "geometry": mapping(simplified)
            }
            features.append(new_feat)

    except Exception:
        pass

    if i % 50 == 0: print(f"   {i}...")

# 3. Save Files
print("3. Saving 'final_israel_map.geojson'...")
with open("final_israel_map.geojson", "w") as f:
    json.dump({"type": "FeatureCollection", "features": features}, f)

print("4. Saving 'final_israel_border.geojson'...")
full_border = unary_union(border_geoms)
with open("final_israel_border.geojson", "w") as f:
    json.dump({
        "type": "FeatureCollection",
        "features": [{"type": "Feature", "properties": {"name": "Border"}, "geometry": mapping(full_border)}]
    }, f)

print("DONE! Please upload the two 'final_' files to your app.")

1. Scanning repository...
2. Processing 259 municipalities...
   0...
   50...
   100...
   150...
   200...
   250...
3. Saving 'final_israel_map.geojson'...
4. Saving 'final_israel_border.geojson'...
DONE! Please upload the two 'final_' files to your app.


In [ ]:
!pip install chardet


In [ ]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from shapely.geometry import Point
import time

def map_settlements():
    # 1. The list of Hebrew names you provided
    yeshuv_klita_values = [
        'שחר', 'חצבה', 'מזכרת בתיה', 'יסוד המעלה', 'אופקים', 'בת שלמה',
        'מטולה', 'יבנאל', 'כפר תבור', 'עתלית', 'נחושה', 'יד רמבם',
        'מרחביה קבוץ', 'ניר עוז', 'אשדוד', 'עין הוד', 'כפר גלעדי',
        'אילת השחר', 'קרית ענבים', 'גניגר', 'יגור', 'מצפה רמון',
        'מאיר שפיה', 'כפר סילבר', 'רמת ישי', 'שריד', 'משמר העמק', 'גבת',
        'עין שמר', 'כפר יהושע', 'שדה יעקב', 'בית זרע', 'גבעת ברנר',
        'תל מונד', 'באר טוביה', 'עינות', 'עין ורד', 'נען', 'תמורים',
        'גן יבנה', 'כפר יונה', 'פרדסיה', 'אביחיל', 'רמת יוחנן',
        'אבן יהודה', 'נעורים', 'אשדות יעקב מאוחד', 'משמר השרון', 'קדימה',
        'מעברות', 'צופית', 'אשדות יעקב אחוד', 'חגלה', 'רמות השבים',
        'גבעת חן', 'מוצא עלית', 'בית צבי', 'כפר הראה', 'עין עירון',
        'גני תקוה', 'שער העמקים', 'יקנעם עלית', 'יקנעם מושבה', 'בית השטה',
        'נתיבות', 'רשפון', 'בית שערים', 'כפר סירקין', 'הזורע', 'ביתן אהרן',
        'ניר דוד', 'כפר שמריהו', 'טירת צבי', 'מולדת', 'עין השופט',
        'מעוז חיים', 'חניתה', 'שדה ורבורג', 'מעלה החמשה', 'בית יהושע',
        'עין המפרץ', 'נוה איתן', 'כפר מסריק', 'דפנה', 'שדה אליהו', 'גשר',
        'כפר נטר', 'שאר ישוב', 'אלומות', 'ניר צבי', 'קבוצת יבנה',
        'רמת השופט', 'איבים', 'גליל ים', 'גבולות', 'רביבים', 'נצנים',
        'רוחמה', 'בית קשת', 'ביריה', 'גזר', 'רמות נפתלי', 'חוקוק',
        'להבות הבשן', 'מכמרת', 'עין הנציב', 'בני דרור', 'כפר מונש', 'בארי',
        'אבן שמואל', 'אורים', 'נוה אילן', 'נאות מרדכי', 'יחיעם', 'צאלים',
        'בני ציון', 'משאבי שדה', 'חרב לאת', 'עין דור', 'כפר הנשיא',
        'אודים', 'נחלים', 'בארות יצחק', 'יזרעאל', 'יפתח', 'שורש',
        'רמת רזיאל', 'הראל', 'צובה', 'בית דגן', 'קרית עקרון',
        'דאלית אל-כרמל', 'עראבה', 'עספיא', 'בוסתן הגליל', 'אזור',
        'גבעת שמש', 'בית גמליאל', 'יד בנימין', 'כפר ראש הנקרה', 'זיקים',
        'לביא', 'סביון', 'ערוגות', 'צפריה', 'מלכיה', 'פלמחים', 'פרוד',
        'ספסופה', 'סתריה', 'רנתיה', 'ברקאי', 'חדיד', 'עין צורים', 'יראון',
        'עמר', 'ברעם', 'מפלסים', 'משמר אילון', 'עין השלשה', 'מעגן',
        'גבעת שמואל', 'היוגב', 'מעגן מיכאל', 'מגן', 'כפר חבד', 'בורגתה',
        'גיאה', 'בית זית', 'עזריה', 'להבות חביבה', 'איל', 'תל קציר',
        'בית הגדי', 'הודיה', 'בית שקמה', 'תעוז', 'שתולים', 'כפר מרדכי',
        'ניר עציון', 'מבוא ביתר', 'אמונים', 'יד נתן', 'עמינדב', 'אחיהוד',
        'גבעת יערים', 'זיתן', 'צלפון', 'אחיעזר', 'זכריה', 'חמד', 'חניאל',
        'שלומי', 'עין יעקב', 'עזריקם', 'מטע', 'ירוחם', 'עזריאל', 'כסופים',
        'דבירה', 'שעלבים', 'כפר אביב', 'נוה ירק', 'מגדל העמק', 'שדה בוקר',
        'גבעת ישעיהו', 'רכסים', 'מבשרת ציון', 'אור עקיבא', 'שדרות',
        'קרית מלאכי', 'רמת אפעל', 'קטורה', 'נוף הגליל', 'מעלות תרשיחא',
        'בני עיש', 'דובב', 'אדמית', 'בית ברל', 'מבועים',
        'בן שמן כפר הנוער', 'כרם יבנה ישיבה', 'מרכז שפירא', 'גני הדר',
        'יודפת', 'צור הדסה', 'לי און', 'לוד נמל תעופה', 'אילות',
        'מעלה גלבע', 'מי עמי', 'קרית יערים טלסטון', 'כרמיאל',
        'מדרשת בן גוריון', 'מבוא מודיעים', 'צור נתן', 'עזר', 'קיסריה',
        'מורשת', 'טל אל', 'אשחר', 'מצפה נטופה', 'בר יוחאי',
        'ניצנה(קהילת חנוך)', 'עדי', 'מודיעין-מכבים-רעות', 'הררית',
        'כוכב יאיר', 'נירית', 'סופה', 'עין הבשור', 'קציר', 'תמרת', 'חריש',
        'כפר ורדים', 'כרמי יוסף', 'מיתר', 'להבים', 'גן נר', 'אבטליון',
        'מירב', 'גבעת אלה', 'כפר חנניה', 'שוהם', 'צורן', 'אלעד',
        'פוריה עלית', 'בת הדר', 'נוף איילון', 'צור יצחק', 'תנובות',
        'בן שמן מושב', 'ברקת', 'עין גדי', 'תדהר', 'טירת כרמל', 'דימונה',
        'קרית טבעון', 'אור יהודה', 'נשר', 'באר יעקב', 'גדרה', 'ערד',
        'אילת', 'בית שמש', 'קרית אונו', 'קרית גת', 'ראש העין', 'רמת השרון',
        'יבנה', 'קרית שמונה', 'ירושלים', 'חברון', 'כפר עציון', 'אלמוג',
        'קדומים', 'אלקנה', 'מגדל עז', 'תקוע', 'כוכב השחר', 'רימונים',
        'אריאל', 'בית אל', 'מצפה יריחו', 'שדמות מחולה', 'אלון שבות',
        'משואה', 'גלגל', 'מעלה אפרים', 'מצפה שלם', 'קרית ארבע',
        'מעלה אדומים', 'עפרה', 'אלעזר', 'כפר אדמים', 'קרני שומרון', 'שילה',
        'חיננית', 'גבעון חדשה', 'אפרתה', 'מעלה מכמש', 'בית אריה', 'ברקן',
        'פסגות', 'ברכה', 'ענב', 'נחל חרמש', 'נוה דניאל', 'נוקדים',
        'גבעת זאב', 'ברוכין', 'קרית נטפים', 'עתניאל', 'יצהר', 'אלפי מנשה',
        'סוסיה', 'אורנית', 'איתמר', 'גבע בנימין', 'עלי', 'פדואל', 'הר אדר',
        'חשמונאים', 'כוכב יעקב', 'ביתר עלית', 'נעלה', 'טלמון', 'נופים',
        'אבני חפץ', 'בת עין', 'רבבה', 'מודיעין עלית', 'גני מודיעין',
        'שער שומרון', 'חיפה', 'קשת', 'גבעת יואב', 'קצרין', 'נוב',
        'תל אביב - יפו', 'בני ברק', 'בת ים', 'גבעתים', 'הרצליה', 'חדרה',
        'חולון', 'טבריה', 'קרית אתא', 'כפר סבא', 'לוד', 'אשקלון',
        'נס ציונה', 'נתניה', 'עכו', 'עפולה', 'פרדס חנה כרכור', 'פתח תקוה',
        'צפת', 'קרית מוצקין', 'ראשון לציון', 'רחובות', 'רמלה', 'רמת גן',
        'רעננה', 'באר שבע', 'נהריה', 'זכרון יעקב', 'יהוד', 'קרית ביאליק',
        'קרית ים', 'הוד השרון', 'בנימינה', 'בית שאן', 'מגן שאול',
        'בית רמון', 'אליפז', 'הרדוף', 'לוטן', 'בית ניר', 'ראש פינה',
        'עוצם', 'אליכין', 'מנחמיה', 'כנרת קבוצה', 'דגניה א', 'כנרת מושבה',
        'מגדל', 'אור הנר', 'נהלל', 'חפצי בה', 'בית אלפא', 'תל עדשים',
        'מזרע', 'רמת רחל', 'דבורה', 'אפיקים', 'רמת הכובש', 'כפר חיים',
        'שושנת העמקים רסקו', 'שפיים', 'בצרון', 'חבצלת השרון', 'גנוסר',
        'עין גב', 'צור משה', 'מעין צבי', 'כפר רופין', 'מסלות', 'מצובה',
        'הגושרים', 'כדורי - ביס חקלאי', 'המעפיל', 'משגב עם', 'רגבה',
        'שובל', 'משמר הנגב', 'נבטים', 'ניר יצחק', 'משמרת', 'ברור חיל',
        'נצר סירני', 'נורדיה', 'טל שחר', 'אבו גוש', 'כפר כמא',
        'בית אלעזרי', 'רבדים', 'צרעה', 'גשר הזיו', 'כרם מהרל', 'בני ראם',
        'נוה אור', 'עשרת', 'מזור', 'צרופה', 'קדרון', 'חצב', 'ארז',
        'גן יאשיה', 'רמות מאיר', 'דור', 'בית מאיר', 'פורת', 'כרמיה',
        'אורה', 'יכיני', 'אחוזם', 'נחל עוז', 'כפר עזה', 'גינתון', 'ישע',
        'אמירים', 'אדרת', 'אלומה', 'עלומים', 'אשלים', 'לוטם', 'טללים',
        'קורנית', 'צביה', 'כליל', 'כרמים', 'תלמי יוסף', 'נתיב העשרה',
        'נופית', 'שני', 'גבעת אבני', 'לפיד', 'מתן', 'בת חפר', 'מרחב עם',
        "ניצן ב'", 'ינון', 'אורות', 'שדי חמד', 'גבעת חיים אחוד', 'להב',
        'מנוחה', 'חצור הגלילית', 'מאור', 'חלמיש', 'אלון מורה', 'הר גילה',
        'פצאל', 'מעלה שומרון', 'מתתיהו', 'שקד', 'מעלה עמוס', 'נילי',
        'מעון', 'עמנואל', 'עלמון', 'תלם', 'נחל נגוהות', 'עלי זהב', 'דלב',
        'כרמי צור', 'צופין', 'חספין', 'אניעם', 'מיצר', 'מורן', 'איתן',
        'לוזית', 'כפר יחזקאל', 'גבע', 'כפר מלל', 'כפר חסידים א', 'נטעים',
        'כפר הס', 'כפר פינס', 'הדר עם', 'דליה', 'שדה נחמיה', 'כפר סאלד',
        'מנרה', 'כפר בלום', 'שמיר', 'גלאון', 'חצרים', 'חצור אשדוד',
        'קלחים', 'סעד', 'העוגן', 'גבים', 'רגבים', 'רמות מנשה', 'געתון',
        'בית העמק', 'כברי', 'מגידו', "ג'לג'וליה", 'גבע כרמל', 'בניה',
        'נוה ימין', 'מגדים', 'גבעת עוז', 'כפר דניאל', 'תפרח', 'שדה אילן',
        'משמר השבעה', 'אליפלט', 'גורן', 'מחסיה', 'יערה', 'גבעת כח', 'ישעי',
        'תלמים', 'בר גיורא', 'אלישמע', 'כפר הריף', 'כפר הנער הדתי',
        'טבעון', 'חרוצים', 'רם און', 'נוה מיכאל', 'כפר מימון', 'יעל',
        'גרופית', 'יד השמונה', 'צוחר', 'כישור', 'מנוף', 'גילון', 'לבון',
        'הילה', 'כחל', 'יתד', 'רקפת', 'לבנים', 'דקל', 'חנתון', 'נוה שלום',
        'נחל שיטים', 'נצני סיני', 'מדרך עוז', 'קרית יערים', 'סגולה',
        'שדה צבי', 'תאשור', 'גבעון', 'מכורה', 'יקיר', 'מבוא חורון',
        'אדורה', 'אודם', 'קלע', 'עין זיוון', 'רמות', 'אחוה', 'אמנון',
        'נטף', 'אilניה', 'גבעת עדה', 'בלפוריה', 'חולדה', 'חבת ציון', 'מעש',
        'חולתה', 'בית יוסף', 'כפר מנחם', 'שרונה', 'שדה יואב',
        'כפר גליקסון', 'בית אורן', 'בית יצחק שער חפר', 'עזוז', 'יד מרדכי',
        'מגל', 'בצרה', 'כפר גלים', 'בית לחם הגלילית', 'החותרים', 'רשפים',
        'בני עטרות', 'גזית', 'בני דרום', 'שפיר', 'ירחיב', 'שואבה',
        'מסלת ציון', 'פדויים', 'מצליח', 'עמיקם', 'רנן', 'יושיביה', 'נחם',
        'כוכב מיכאל', 'בית רבן', 'אלוני יצחק', 'בית חשמונאי', 'מעגלים',
        'צוקי ים', 'פוריה נוה עובד', 'אומן', 'ספיר', 'אלון הגליל',
        'הושעיה', 'שזפון', 'חלוץ', 'כמהין', 'אבשלום', 'ארסוף', 'אירוס',
        'שמשית', 'גבעת חביבה', 'ניר עקיבא', 'דישון', 'תמר', 'יפית', 'קדר',
        'אלרום', 'קדמת צבי', 'סמר', 'יהל', 'שילת', 'ניר חן', 'רמת דוד',
        'עין החורש', 'כפר בילו', 'כפר ויתקין', 'משמרות', 'גן השומרון',
        'כפר המכבי', 'שער הגולן', 'שבי ציון', 'אלונים', 'בית הלל',
        'רמת צבי', 'גברעם', 'חפץ חיים', 'יקום', 'יסודות', 'בית קמה',
        'נירים', 'בית עריף', 'אשתאול', 'יציץ', 'בטחה', 'בית חנניה',
        'נצני עוז', 'רחוב', 'שדה אליעזר', 'יטבתה', 'צפרירים', 'תושיה',
        'תובל', 'עדן', 'כמון', 'צורית', 'שדי אברהם', 'בת חן', 'בחן',
        'ראש צורים', 'נעמי', 'מעלה לבונה', 'כפר האורנים', 'אלי על',
        'כפר רות', 'כורזים', 'באר אורה', 'לכיש', 'חופית', 'ברק', 'אלישיב',
        'אושה', 'דורות', 'שרשרת', 'שוקדה', 'סער', 'ריחאניה', 'מבקיעים',
        'לוחמי הגיטאות', 'שער אפרים', 'אליקים', 'עין אילה', 'בארותים',
        'מגשימים', 'ברכיה', 'ינוב', 'בית עזרא', 'נחשון', 'תרום', 'גנות',
        'פתחיה', 'געש', 'גבעת השלשה', 'גאולי תימן מושב', 'בית חרות',
        'עין שריד', 'תלמי אליהו', 'אביעזר', 'אביבים', 'יעד', 'עמוקה',
        'נוה חריף', 'נווה זיו', 'אחוזת ברק', 'שדה יצחק', 'כפר בן נון',
        'בית חורון', 'ורד יריחו', 'שמעה', 'חד נס', 'נוה אטיב', 'דגניה ב',
        'יפעת', 'גן שלמה', 'כפר החורש', 'בית ינאי', 'הזורעים', 'חורשים',
        'כפר אוריה', 'עיילבון', 'סאסא', 'מירון', 'צפורי', 'תקומה',
        'נחשונים', 'חגור', 'קוממיות', 'צוריאל', 'נוה מבטח', 'ישרש',
        'גאליה', 'שגב', 'שדה נצן', 'אבן מנחם', 'נטועה', 'לפידות', 'חרשים',
        'יובלים', 'רתמים', 'בני דקלים', 'כפר תפוח', 'נחליאל', 'נצרת',
        'שפרעם', 'עין תמר', 'שדה משה', 'נוגה', 'מרחביה מושב', 'חרות',
        'גבעת חיים מאוחד', 'דן', 'עמיר', 'כפר ורבורג', 'ניר עם',
        'עין העמק', 'גלעד (אבן יצחק)', 'כפר קיש', 'נחשולים', 'מעונה',
        'בצת', 'לימן', 'הסוללים', 'אומץ', 'בן עמי', 'עולש', 'יד חנה',
        'משגב דב', 'בית נחמיה', 'משען', 'נס הרים', 'בקוע',
        'פוריה כפר עבודה', 'פלך', 'מחנה יתיר', 'שורשים', 'טירה',
        'שבי שומרון', 'ייטב', 'מצדות יהודה', 'כפר חרוב', 'יונתן',
        'בני יהודה', 'אפיק', 'נאות הגולן'
    ]

    # Filter out None or empty values
    yeshuv_klita_values = [x for x in yeshuv_klita_values if x]

    # 2. Initialize Geocoding
    # We use OSM Nominatim. You might need to change 'user_agent' if you make many requests.
    geolocator = Nominatim(user_agent="israel_mapper_script")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

    print(f"Starting to map {len(yeshuv_klita_values)} settlements. This may take a while...")

    locations = []

    for i, name in enumerate(yeshuv_klita_values):
        try:
            # We append ", Israel" to help the geocoder context
            search_query = f"{name}, Israel"
            location = geocode(search_query)

            if location:
                locations.append({
                    'hebrew_name': name,
                    'lat': location.latitude,
                    'lon': location.longitude,
                    'address': location.address
                })
            else:
                print(f"Could not find coordinates for: {name}")
                locations.append({'hebrew_name': name, 'lat': None, 'lon': None})

        except Exception as e:
            print(f"Error mapping {name}: {e}")
            locations.append({'hebrew_name': name, 'lat': None, 'lon': None})

        if i % 10 == 0:
            print(f"Processed {i}/{len(yeshuv_klita_values)}...")

    # Create DataFrame from results
    df_locs = pd.DataFrame(locations)

    # Filter only those found
    df_found = df_locs.dropna(subset=['lat', 'lon'])
    print(f"Successfully geocoded {len(df_found)} out of {len(yeshuv_klita_values)} names.")

    # 3. Load Map and Perform Spatial Join
    print("Loading map and performing spatial join...")
    map_gdf = gpd.read_file("final_israel_map.geojson")

    # Create a GeoDataFrame for the points
    gdf_points = gpd.GeoDataFrame(
        df_found,
        geometry=gpd.points_from_xy(df_found.lon, df_found.lat),
        crs="EPSG:4326" # WGS84 standard lat/lon
    )

    # Ensure map CRS matches
    if map_gdf.crs != gdf_points.crs:
        map_gdf = map_gdf.to_crs(gdf_points.crs)

    # Spatial Join: Find which polygon each point is inside
    joined = gpd.sjoin(gdf_points, map_gdf, how="left", predicate="within")

    # 4. Final Cleanup and Save
    # We select the Hebrew name and the 'name' column from the map (which is the district/council)
    final_output = joined[['hebrew_name', 'name']].rename(columns={'name': 'mapped_district'})

    # Merge back with original list to keep everything (even unmapped ones)
    df_all = pd.DataFrame({'hebrew_name': yeshuv_klita_values})
    final_dataset = df_all.merge(final_output, on='hebrew_name', how='left')

    print("Sample of the results:")
    print(final_dataset.head(10))

    final_dataset.to_csv("mapped_yeshuvim.csv", index=False, encoding='utf-8-sig')
    print("Done! Saved to 'mapped_yeshuvim.csv'")

if __name__ == "__main__":
    map_settlements()

Starting to map 838 settlements. This may take a while...
Processed 0/838...
Processed 10/838...
Could not find coordinates for: מרחביה קבוץ
Processed 20/838...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/usr/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ssl.py", line

Processed 30/838...
Could not find coordinates for: תמורים
Processed 40/838...
Processed 50/838...
Could not find coordinates for: מוצא עלית
Could not find coordinates for: כפר הראה
Processed 60/838...
Could not find coordinates for: יקנעם עלית
Could not find coordinates for: בית השטה
Processed 70/838...
Could not find coordinates for: מעלה החמשה
Processed 80/838...
Processed 90/838...
Could not find coordinates for: נצנים
Processed 100/838...
Could not find coordinates for: מכמרת
Processed 110/838...


Processed 120/838...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/usr/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ssl.py", line

...

Processed 520/838...
Could not find coordinates for: כפר מלל
Processed 530/838...


Processed 540/838...
Processed 550/838...
Processed 560/838...


Processed 570/838...
Could not find coordinates for: כפר הנער הדתי
Could not find coordinates for: נוה מיכאל
Processed 580/838...
Processed 590/838...
Could not find coordinates for: נחל שיטים
Could not find coordinates for: נצני סיני
Processed 600/838...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/usr/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ssl.py", line

Processed 610/838...
Could not find coordinates for: אilניה
Processed 620/838...
Processed 630/838...
Processed 640/838...
Processed 650/838...
Could not find coordinates for: פוריה נוה עובד
Processed 660/838...
Could not find coordinates for: קדר
Processed 670/838...
Processed 680/838...
Processed 690/838...
Could not find coordinates for: נצני עוז
Processed 700/838...
Processed 710/838...
Could not find coordinates for: ראש צורים
Could not find coordinates for: מעלה לבונה
Could not find coordinates for: כפר האורנים
Processed 720/838...
Processed 730/838...
Could not find coordinates for: בארותים
Processed 740/838...
Could not find coordinates for: גבעת השלשה
Could not find coordinates for: גאולי תימן מושב
Processed 750/838...
Could not find coordinates for: נוה חריף
Processed 760/838...
Could not find coordinates for: ורד יריחו
Could not find coordinates for: נוה אטיב
Processed 770/838...
Could not find coordinates for: צפורי
Processed 780/838...
Could not find coordinates for: נוה מ

In [ ]:
import pandas as pd

# 1. טעינת קובץ התוצאות שיצרת
df = pd.read_csv("mapped_yeshuvim.csv", encoding='utf-8-sig')

# 2. סינון השורות שבהן אין שיוך למחוז (ערכי NaN)
missing = df[df['mapped_district'].isna()]

# 3. קבלת רשימת השמות הייחודיים (ללא כפילויות)
missing_names = missing['hebrew_name'].unique()

print(f"סה''כ שמות שלא נמצאו: {len(missing_names)}")
print("--- רשימת הישובים החסרים ---")
for name in missing_names:
    print(name)

# אופציונלי: שמירת הרשימה החסרה לקובץ נפרד כדי לעבור עליה ידנית
# pd.DataFrame(missing_names, columns=['missing_name']).to_csv("missing_names.csv", index=False, encoding='utf-8-sig')

סה''כ שמות שלא נמצאו: 78
--- רשימת הישובים החסרים ---
מרחביה קבוץ
תמורים
מוצא עלית
כפר הראה
יקנעם עלית
בית השטה
מעלה החמשה
נצנים
מכמרת
עין השלשה
כרם יבנה ישיבה
לוד נמל תעופה
מעלה גלבע
קרית יערים טלסטון
ניצנה(קהילת חנוך)
בן שמן מושב
מצפה יריחו
שדמות מחולה
אלון שבות
מעלה אפרים
מצפה שלם
כפר אדמים
קרני שומרון
גבעון חדשה
מעלה מכמש
נחל חרמש
נוה דניאל
ברוכין
קרית נטפים
אלפי מנשה
גבע בנימין
פדואל
הר אדר
כוכב יעקב
ביתר עלית
אבני חפץ
רבבה
מודיעין עלית
שער שומרון
גבעתים
כנרת קבוצה
שושנת העמקים רסקו
בצרון
כדורי - ביס חקלאי
כרם מהרל
אלון מורה
מעלה שומרון
נחל נגוהות
עלי זהב
דלב
כרמי צור
צופין
כפר מלל
כפר הנער הדתי
נוה מיכאל
נחל שיטים
נצני סיני
אilניה
פוריה נוה עובד
קדר
נצני עוז
ראש צורים
מעלה לבונה
כפר האורנים
בארותים
גבעת השלשה
גאולי תימן מושב
נוה חריף
ורד יריחו
נוה אטיב
צפורי
נוה מבטח
שדה נצן
כפר תפוח
מחנה יתיר
שבי שומרון
ייטב
מצדות יהודה


In [1]:
!pip install pandas geopandas geopy rtree

In [4]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import os

def fix_missing_mappings():
    # ---------------------------------------------------------
    # 1. SETUP & CONFIGURATION
    # ---------------------------------------------------------
    CSV_PATH = "mapped_yeshuvim.csv"
    GEOJSON_PATH = "final_israel_map.geojson"

    # Dictionary of manual corrections (Your Input -> Searchable Name)
    NAME_CORRECTIONS = {
        'אilניה': 'אילניה',
        'גבעתים': 'גבעתיים',
        'לוד נמל תעופה': 'נמל תעופה בן גוריון',
        'נצנים': 'ניצנים',
        'מכמרת': 'מכמורת',
        'יקנעם עלית': 'יקנעם עילית',
        'בית השטה': 'בית השיטה',
        'מעלה החמשה': 'מעלה החמישה',
        'קרית יערים טלסטון': 'קריית יערים',
        'ניצנה(קהילת חנוך)': 'ניצנה',
        'בן שמן מושב': 'בן שמן',
        'גבעון חדשה': 'גבעון החדשה',
        'כנרת קבוצה': 'קבוצת כנרת',
        'שושנת העמקים רסקו': 'שושנת העמקים',
        'כדורי - ביס חקלאי': 'כדורי',
        'כרם מהרל': 'כרם מהר"ל',
        'נחל נגוהות': 'נגוהות',
        'נחל חרמש': 'חרמש',
        'נחל שיטים': 'שיטים',
        'קרית נטפים': 'קריית נטפים',
        'מצדות יהודה': 'בית יתיר',
        'מודיעין עלית': 'מודיעין עילית',
        'ביתר עלית': 'ביתר עילית',
        'צופין': 'צופים',
        'מרחביה קבוץ': 'קיבוץ מרחביה',
        'כפר הנער הדתי': 'כפר הנוער הדתי',
        'פוריה נוה עובד': 'פורייה - נווה עובד',
        'גבעת השלשה': 'גבעת השלושה',
        'מצפה שלם': 'מצפה שלם',
        'נצני סיני': 'ניצני סיני',
        'נצני עוז': 'ניצני עוז',
        'נוה דניאל': 'נווה דניאל',
        'נוה מיכאל': 'נווה מיכאל',
        'נוה חריף': 'נווה חריף',
        'נוה אטיב': 'נווה אטי"ב',
        'נוה מבטח': 'נווה מבטח',
        'שדה נצן': 'שדה ניצן',
        'צפורי': 'ציפורי',
        'בארותים': 'בארותיים',
        'גאולי תימן מושב': 'גאולי תימן',
        'תמורים': 'תימורים',
        'כפר הראה': 'כפר הרא"ה',
        'עין השלשה': 'עין השלושה',
        'כרם יבנה ישיבה': 'כרם ביבנה',
        'מעלה גלבע': 'מעלה גלבוע',
        'מצפה יריחו': 'מצפה יריחו',
        'שדמות מחולה': 'שדמות מחולה',
        'אלון שבות': 'אלון שבות',
        'מעלה אפרים': 'מעלה אפריים',
        'כפר אדמים': 'כפר אדומים',
        'קרני שומרון': 'קרני שומרון',
        'מעלה מכמש': 'מעלה מכמש',
        'ברוכין': 'ברוכין',
        'אלפי מנשה': 'אלפי מנשה',
        'גבע בנימין': 'גבע בנימין',
        'פדואל': 'פדואל',
        'הר אדר': 'הר אדר',
        'כוכב יעקב': 'כוכב יעקב',
        'אבני חפץ': 'אבני חפץ',
        'רבבה': 'רבבה',
        'שער שומרון': 'שערי תקווה',
        'בצרון': 'ביצרון',
        'אלון מורה': 'אלון מורה',
        'מעלה שומרון': 'מעלה שומרון',
        'עלי זהב': 'עלי זהב',
        'דלב': 'דולב',
        'כרמי צור': 'כרמי צור',
        'כפר מלל': 'כפר מל"ל',
        'קדר': 'קידר',
        'ראש צורים': 'ראש צורים',
        'מעלה לבונה': 'מעלה לבונה',
        'כפר האורנים': 'כפר אורנים',
        'ורד יריחו': 'ורד יריחו',
        'כפר תפוח': 'כפר תפוח',
        'מחנה יתיר': 'יתיר',
        'שבי שומרון': 'שבי שומרון',
        'ייטב': 'ייט"ב'
    }

    # ---------------------------------------------------------
    # 2. LOAD DATA
    # ---------------------------------------------------------
    if not os.path.exists(CSV_PATH):
        print(f"Error: {CSV_PATH} not found.")
        return

    print("Loading existing CSV...")
    df = pd.read_csv(CSV_PATH)

    # Identify rows that need processing
    # We look for rows where 'mapped_district' is missing (NaN)
    if 'mapped_district' not in df.columns:
        print("Column 'mapped_district' missing. Creating it.")
        df['mapped_district'] = None

    # Filter: Rows where district is missing
    missing_mask = df['mapped_district'].isna() | (df['mapped_district'] == '')
    missing_df = df[missing_mask].copy()

    if missing_df.empty:
        print("No missing mappings found! Exiting.")
        return

    print(f"Found {len(missing_df)} settlements to fix.")

    # ---------------------------------------------------------
    # 3. GEOCODING (Only for missing items)
    # ---------------------------------------------------------
    geolocator = Nominatim(user_agent="israel_fixer_script")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.0)

    results = []

    print("Starting geocoding process...")
    for index, row in missing_df.iterrows():
        original_name = row['hebrew_name']

        # Determine search query (use dictionary if available, else original)
        search_name = NAME_CORRECTIONS.get(original_name, original_name)
        query = f"{search_name}, Israel"

        try:
            location = geocode(query)
            if location:
                results.append({
                    'index': index, # Keep original index to update later
                    'lat': location.latitude,
                    'lon': location.longitude
                })
            else:
                print(f"  [X] Failed: {original_name} (Searched: {search_name})")

        except Exception as e:
            print(f"  [!] Error on {original_name}: {e}")

    if not results:
        print("No new coordinates found. Exiting.")
        return

    print(f"Successfully geocoded {len(results)} settlements.")

    # ---------------------------------------------------------
    # 4. SPATIAL JOIN
    # ---------------------------------------------------------
    print("Loading GeoJSON map...")
    map_gdf = gpd.read_file(GEOJSON_PATH)

    # Convert results to GeoDataFrame
    df_results = pd.DataFrame(results)
    gdf_points = gpd.GeoDataFrame(
        df_results,
        geometry=gpd.points_from_xy(df_results.lon, df_results.lat),
        crs="EPSG:4326"
    )

    # Ensure CRS matches
    if map_gdf.crs != gdf_points.crs:
        map_gdf = map_gdf.to_crs(gdf_points.crs)

    # Perform Spatial Join
    # We want to find which polygon (district) contains the point
    joined = gpd.sjoin(gdf_points, map_gdf, how="left", predicate="within")

    # ---------------------------------------------------------
    # 5. UPDATE ORIGINAL CSV
    # ---------------------------------------------------------
    print("Updating original records...")

    # We update the main 'df' using the 'index' we saved earlier
    for _, row in joined.iterrows():
        original_idx = row['index']
        district_name = row['name'] # Adjust 'name' if your GeoJSON uses a different column for district name

        if pd.notna(district_name):
            df.at[original_idx, 'mapped_district'] = district_name
            print(f"  -> Mapped index {original_idx} to {district_name}")

    # ---------------------------------------------------------
    # 6. SAVE
    # ---------------------------------------------------------
    df.to_csv("mapped_yeshuvim_fixed.csv", index=False, encoding='utf-8-sig')
    print("Done! Saved updated file to 'mapped_yeshuvim_fixed.csv'")

    # Optional: Overwrite original if you prefer
    # df.to_csv(CSV_PATH, index=False, encoding='utf-8-sig')

if __name__ == "__main__":
    fix_missing_mappings()

Loading existing CSV...
Found 78 settlements to fix.
Starting geocoding process...
  [X] Failed: מוצא עלית (Searched: מוצא עלית)
  [X] Failed: מצפה יריחו (Searched: מצפה יריחו)
  [X] Failed: שדמות מחולה (Searched: שדמות מחולה)
  [X] Failed: אלון שבות (Searched: אלון שבות)
  [X] Failed: מעלה אפרים (Searched: מעלה אפריים)
  [X] Failed: מצפה שלם (Searched: מצפה שלם)
  [X] Failed: כפר אדמים (Searched: כפר אדומים)
  [X] Failed: קרני שומרון (Searched: קרני שומרון)
  [X] Failed: גבעון חדשה (Searched: גבעון החדשה)
  [X] Failed: מעלה מכמש (Searched: מעלה מכמש)
  [X] Failed: נוה דניאל (Searched: נווה דניאל)
  [X] Failed: ברוכין (Searched: ברוכין)
  [X] Failed: קרית נטפים (Searched: קריית נטפים)
  [X] Failed: אלפי מנשה (Searched: אלפי מנשה)
  [X] Failed: גבע בנימין (Searched: גבע בנימין)
  [X] Failed: פדואל (Searched: פדואל)
  [X] Failed: הר אדר (Searched: הר אדר)
  [X] Failed: כוכב יעקב (Searched: כוכב יעקב)


KeyboardInterrupt: 

In [8]:
import sys
import pandas as pd
from pathlib import Path

def remove_municipalities(csv_path):
    csv_path = Path(csv_path)

    # Read CSV
    df = pd.read_csv(csv_path)

    # Filter out rows where mapped_district == 'municipalities'
    df_filtered = df[df["mapped_district"] != "municipalities"]

    # Save to a new file (same name with _cleaned suffix)
    output_path = csv_path.with_name(csv_path.stem + "_cleaned.csv")
    df_filtered.to_csv(output_path, index=False)

    print(f"Cleaned file saved to: {output_path}")
    print(f"Removed {len(df) - len(df_filtered)} rows.")

if __name__ == "__main__":
    remove_municipalities('/content/mapped_yeshuvim_fixed.csv')


Cleaned file saved to: /content/mapped_yeshuvim_fixed_cleaned.csv
Removed 0 rows.


Adding last 33 remaining mappings

In [7]:
import pandas as pd
import re

CSV_PATH = "/content/mapped_yeshuvim_fixed.csv"

manual_mappings = {
    'מוצא עלית': 'jerusalem_yerushalayim',
    'מצפה יריחו': 'mate_binyamin',
    'שדמות מחולה': 'bikat_hayarden',
    'אלון שבות': 'gush_etzion',
    'מעלה אפרים': 'maale_efraim',
    'מצפה שלם': 'megilot',
    'כפר אדמים': 'mate_binyamin',
    'קרני שומרון': 'karnei_shomron',
    'גבעון חדשה': 'mate_binyamin',
    'מעלה מכמש': 'mate_binyamin',
    'נוה דניאל': 'gush_etzion',
    'ברוכין': 'shomron',
    'קרית נטפים': 'shomron',
    'אלפי מנשה': 'alfei_menashe',
    'גבע בנימין': 'mate_binyamin',
    'פדואל': 'shomron',
    'הר אדר': 'har_adar',
    'כוכב יעקב': 'mate_binyamin',
    'אבני חפץ': 'shomron',
    'רבבה': 'shomron',
    'מודיעין עלית': 'modiin_ilit',
    'אלון מורה': 'shomron',
    'מעלה שומרון': 'shomron',
    'עלי זהב': 'shomron',
    'כרמי צור': 'gush_etzion',
    'פוריה נוה עובד': 'emek_hayarden',
    'ראש צורים': 'gush_etzion',
    'מעלה לבונה': 'mate_binyamin',
    'כפר האורנים': 'mate_binyamin',
    'ורד יריחו': 'megilot',
    'כפר תפוח': 'shomron',
    'שבי שומרון': 'shomron',
    'ייטב': 'bikat_hayarden'
}

YESHUV_COL = "hebrew_name"
DISTRICT_COL = "mapped_district"

def normalize_hebrew(s):
    if pd.isna(s):
        return s
    s = str(s)
    s = s.replace("\u00A0", " ")     # non-breaking space
    s = re.sub(r"\s+", " ", s)       # collapse spaces
    return s.strip()

# Load CSV
df = pd.read_csv(CSV_PATH)

# Normalize yeshuv column
df["_yeshuv_norm"] = df[YESHUV_COL].apply(normalize_hebrew)

added = []
updated = []

for yeshuv, district in manual_mappings.items():
    norm = normalize_hebrew(yeshuv)
    mask = df["_yeshuv_norm"] == norm

    if mask.any():
        df.loc[mask, DISTRICT_COL] = district
        updated.append(yeshuv)
    else:
        added.append(yeshuv)
        df = pd.concat(
            [df, pd.DataFrame([{YESHUV_COL: yeshuv, DISTRICT_COL: district, "_yeshuv_norm": norm}])],
            ignore_index=True
        )

# Clean up helper column
df.drop(columns="_yeshuv_norm", inplace=True)

# Save
df.to_csv(CSV_PATH, index=False)

print(f"Updated {len(updated)} existing yeshuvim.")
print(f"Added {len(added)} new yeshuvim.")
if added:
    print("Added:", added)


Updated 33 existing yeshuvim.
Added 0 new yeshuvim.
